# 1. Intro - imports

In [ ]:
! qi login "https://api.quantum-inspire.com"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from qiskit import QuantumCircuit, transpile
from qiskit_quantuminspire.qi_provider import QIProvider
from qi_utilities.utility_functions.circuit_modifiers import apply_readout_circuit
from qi_utilities.utility_functions.raw_data_processing import get_multi_counts, get_multi_probs, observable_expectation_values_Z_basis
from qi_utilities.utility_functions.readout_correction import (split_raw_shots, extract_ro_assignment_matrix, plot_ro_assignment_matrix,
                                                               get_ro_corrected_multi_probs, measure_ro_assignment_matrix)
from qi_utilities.utility_functions.data_handling import StoreProjectRecord, RetrieveProjectRecord

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
provider = QIProvider()

In [ ]:
provider.backends()

In [ ]:
backend_name = "Tuna-9"
backend = provider.get_backend(name=backend_name)

In [ ]:
backend.coupling_map.draw()

# 2. VQE workspace

In [ ]:
import qiskit.quantum_info as qi
from qi_utilities.algorithms.vqe import construct_hva_circuit, make_ansatz, hardware_efficient_vqe

In [ ]:
nr_qubits = 2
J = 1 # in arbitrary units

In [ ]:
pauli_terms = ['XX', 'YY', 'ZZ']
pauli_coefficients = [J, J, J]

hamiltonian_operator = qi.SparsePauliOp(pauli_terms, pauli_coefficients)
print(hamiltonian_operator)

In [ ]:
qc = construct_hva_circuit(initial_state = '01',
                           hamiltonian = hamiltonian_operator,
                           repetitions = 1)

In [ ]:
qc.draw('mpl')

In [ ]:
qc = construct_hva_circuit(initial_state = '01',
                           hamiltonian = hamiltonian_operator,
                           repetitions = 1)
initial_point = np.random.random(qc.num_parameters)
qc_instance = make_ansatz(qc, initial_point)

In [ ]:
superconducting_basis_gates = ['id', 'z', 's', 'sdg', 't', 'tdg', 'x', 'rx', 'y', 'ry', 'cz', 'delay', 'reset']
qubit_list = [0, 2]

qc_transpiled = transpile(qc_instance,
                          backend,
                          initial_layout=qubit_list,
                          basis_gates=superconducting_basis_gates)

In [ ]:
initial_point

In [ ]:
qc_transpiled.draw('mpl')

In [ ]:
nr_shots = 2**11 # NOTE: adjust accordingly in case the memory limitation of the
                 # readout instruments is exceeded
job = backend.run(qc_transpiled, shots=nr_shots, memory=True)
result = job.result(timeout = 600)
StoreProjectRecord(job)

# Assembling everything

In [ ]:
nr_qubits = 2
J = 1 # in arbitrary units

pauli_terms = ['XX', 'YY', 'ZZ']
pauli_coefficients = [J, J, J]
hamiltonian_operator = qi.SparsePauliOp(pauli_terms, pauli_coefficients)

superconducting_basis_gates = ['id', 'z', 's', 'sdg', 't', 'tdg', 'x', 'rx', 'y', 'ry', 'cz', 'delay', 'reset']
qubit_list = [0, 2]

In [ ]:
def translate_to_Z_basis(pauli_term: str):

    pauli_term = pauli_term.replace("X", "Z")
    pauli_term = pauli_term.replace("Y", "Z")
    return pauli_term

def calculate_energy(result,
                     hamiltonian,
                     qubit_list,
                     ro_assignment_matrix):
    
    raw_data_shots = result.get_memory()
    raw_data_counts = get_multi_counts(raw_data_shots, len(qubit_list))
    raw_data_probs = get_multi_probs(raw_data_counts)
    ro_corrected_probs = get_ro_corrected_multi_probs(raw_data_probs, ro_assignment_matrix, qubit_list)

    output_energy = 0
    for term_idx in range(len(hamiltonian)):

        pauli_term = hamiltonian[term_idx].paulis[0].to_label()
        pauli_term_in_Z = translate_to_Z_basis(pauli_term)
        observable_value = observable_expectation_values_Z_basis([ro_corrected_probs[term_idx]], pauli_term_in_Z)[0]
        coeff = np.real(hamiltonian[term_idx].coeffs[0])
        output_energy += coeff * observable_value

    print(output_energy)
    return output_energy

In [ ]:
def run_vqe(var_parameters,
            qc_parameterized,
            hamiltonian,
            backend,
            qubit_list,
            basis_gates,
            ro_assignment_matrix,
            nr_shots = 2**11):
    
    qc_instance = make_ansatz(qc_parameterized, var_parameters)
    qc_instance_transpiled = transpile(qc_instance,
                              backend,
                              initial_layout=qubit_list,
                              basis_gates=basis_gates)
    
    job = backend.run(qc_instance_transpiled, shots=nr_shots, memory=True)
    result = job.result(timeout = 600)
    StoreProjectRecord(job, silent=True)

    return calculate_energy(result, hamiltonian, qubit_list, ro_assignment_matrix)

In [ ]:
from scipy.optimize import minimize

qc_parameterized = construct_hva_circuit(initial_state = '01',
                                         hamiltonian = hamiltonian_operator,
                                         repetitions = 1)
initial_point = np.random.uniform(
    low=-np.pi,
    high=np.pi,
    size=qc_parameterized.num_parameters
)
ro_assignment_matrix = measure_ro_assignment_matrix(backend, qubit_list)

result = minimize(
            run_vqe,
            x0=initial_point,
            args=(qc_parameterized,
                  hamiltonian_operator,
                  backend,
                  qubit_list,
                  superconducting_basis_gates,
                  ro_assignment_matrix),
            method="SLSQP",
            bounds=[(-np.pi, np.pi)] * qc_parameterized.num_parameters,
            options={"eps": 0.1 * np.pi,
                     "maxiter": 200}
        )

In [ ]:
from qiskit_algorithms.optimizers import SPSA

qc_parameterized = construct_hva_circuit(initial_state = '01',
                                         hamiltonian = hamiltonian_operator,
                                         repetitions = 1)
initial_point = np.random.uniform(
    low=-np.pi,
    high=np.pi,
    size=qc_parameterized.num_parameters
)

SPSA_optimizer = SPSA(
    maxiter=200,
    perturbation=0.1 * np.pi,
    learning_rate=0.05 * np.pi
)
wrapped_func = SPSA_optimizer.wrap_function(run_vqe,
                             args=(qc_parameterized,
                                   hamiltonian_operator,
                                   backend,
                                   qubit_list,
                                   superconducting_basis_gates,
                                   ro_assignment_matrix))

ro_assignment_matrix = measure_ro_assignment_matrix(backend, qubit_list)
result = SPSA_optimizer.minimize(wrapped_func,
                                 x0 = initial_point,
                                 bounds=[(-np.pi, np.pi)] * qc_parameterized.num_parameters)